In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

RSEED = 42

In [5]:
X_train = pd.read_csv("data/x-train.csv")
y_train = pd.read_csv("data/y-train.csv")

X_test = pd.read_csv("data/x-test.csv")
y_test = pd.read_csv("data/y-test.csv")

In [11]:
# Implementation of Stacking in Scikit-Learn
estimators = [
    ('dt', DecisionTreeClassifier(random_state = RSEED)),
    ('knn', KNeighborsClassifier()),
    ('rf', RandomForestClassifier(n_estimators=100,random_state = RSEED,max_features = 'sqrt',n_jobs=-1, verbose = 1))
]

clf = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)


/Users/lee/neuefische/ds-ml-project/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lee/neuefische/ds-ml-project/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.0s 

0.9990069513406157

In [12]:
stack_y_pred = clf.predict(X_test)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [13]:
print("------"*10)
print(classification_report(y_test, stack_y_pred))
print("------"*10)

# F1-score on test set
print("F1-score:", f1_score(y_test, stack_y_pred, average='binary'))
print("------"*10)


------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19097
           1       0.77      0.67      0.72        36

    accuracy                           1.00     19133
   macro avg       0.89      0.83      0.86     19133
weighted avg       1.00      1.00      1.00     19133

------------------------------------------------------------
F1-score: 0.7164179104477612
------------------------------------------------------------


In [14]:
# Implementation of Stacking in Scikit-Learn
estimators = [
    ('dt', DecisionTreeClassifier(random_state = RSEED)),
    ('ada', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),n_estimators=200)),
    ('rf', RandomForestClassifier(n_estimators=100,random_state = RSEED,max_features = 'sqrt',n_jobs=-1, verbose = 1))
]

clf = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)


/Users/lee/neuefische/ds-ml-project/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lee/neuefische/ds-ml-project/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.4s 

0.9991637484973606

In [15]:
stack_y_pred = clf.predict(X_test)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [16]:
print("------"*10)
print(classification_report(y_test, stack_y_pred))
print("------"*10)

# F1-score on test set
print("F1-score:", f1_score(y_test, stack_y_pred, average='binary'))
print("------"*10)

------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19097
           1       0.79      0.75      0.77        36

    accuracy                           1.00     19133
   macro avg       0.90      0.87      0.89     19133
weighted avg       1.00      1.00      1.00     19133

------------------------------------------------------------
F1-score: 0.7714285714285715
------------------------------------------------------------
